In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-sentiment/training.1600000.processed.noemoticon.csv


In [48]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [49]:
df=pd.read_csv('/kaggle/input/twitter-sentiment/training.1600000.processed.noemoticon.csv',delimiter=",",encoding='ISO-8859-1')

In [50]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [51]:
df.columns=['Polarity','Id','Time','Query',"Twitter_id",'Text']

In [52]:
df.head()

,Polarity,Id,Time,Query,Twitter_id,Text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [53]:
df.info()
# since the data is too large for us to process as would take a while during processing so we limit the data to half a million of rows("500,000" of data)
df=df.head(500000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Polarity    1599999 non-null  int64 
 1   Id          1599999 non-null  int64 
 2   Time        1599999 non-null  object
 3   Query       1599999 non-null  object
 4   Twitter_id  1599999 non-null  object
 5   Text        1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [54]:
df.isnull().sum()
df=df.dropna()
df.isnull().sum()
df['Text'] = df['Text'].fillna('')
df.head()

,Polarity,Id,Time,Query,Twitter_id,Text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [55]:
stopwords= set(stopwords.words('english'))
stemmer=PorterStemmer()
def tf(text):
    text=text.lower()
    words=word_tokenize(text)
    filtered = [stemmer.stem(word.lower()) for word in words if word.isalnum() and word.lower() not in stopwords]
    return ' '.join(filtered)
df['Text'] = df['Text'].apply(tf)

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [59]:
X=df['Text']
y=df['Polarity']
print(X,y)

0         upset ca updat facebook text might cri result ...
1         kenichan dive mani time ball manag save 50 res...
2                           whole bodi feel itchi like fire
3                          nationwideclass behav mad ca see
4                                       kwesidei whole crew
                                ...                        
499995           heyitsmad idea use web p tweetdeck wo work
499996            ww work gain back probabl put chip anyway
499997                            dolphinn sorri mommi miss
499998       terribl headach last night weight decreas aaah
499999                                           cant sleep
Name: Text, Length: 500000, dtype: object 0         0
1         0
2         0
3         0
4         0
         ..
499995    0
499996    0
499997    0
499998    0
499999    0
Name: Polarity, Length: 500000, dtype: int64


In [60]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.25,random_state=42)

In [61]:
vec=TfidfVectorizer(max_features=5000)
X_train_tfidf=vec.fit_transform(X_train)
#x = x.reshape(-1,1)
X_test_tfidf=vec.transform(X_test)

In [ ]:
model=LogisticRegression()
#model2=
#model2.fit(X_train_tfidf, y_train)
model.fit(X_train_tfidf, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [ ]:
y_pred

#### Importing SentimentIntensityAnalyzer(SIA)

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment importSentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia=SentimentIntensityAnalyzer()

In [ ]:
sia.polarity_scores("I am so sad")

Quick EDA

In [ ]:
ax= df['Polarity'].value_counts().sort_index()\
    .plot(kind="bar",
          title="Polarity by Count",
          figsize=(10,6))
ax.set_xlabel()
plt.show